# Parsing Journal of Cognitive Science article data Arthur already scraped (see GDrive)
Aim 1: Parse author & affiliation information for each article.

###### Tentative plans

LOOP THROUGH ALL ROWS: 
- Separate the author and affiliation data
- Create new row for each author in that paper, and denote whether that author was a corresponding author. 
- Also, keep the article name and index (row number) of the article in the original dataset.

In [ ]:
import pandas as pd
import string as st

import matplotlib
import numpy as np
from datascience import *

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
# Load data CSV -> datascience Table

data_file = '../data/180213_cogsci_journal_unprocessed.csv'
df_pd = pd.read_csv(data_file)

#df = Table.read_table(data_file)
df_ad = df_pd['authors'][0:]
df_pd

,article_index,article_name,year,volume,issue,pages,month,doi,article_type,authors,...,references_journals,references_nonjournal,references_titles_all,references_same_number,references_titles_journal,citedby,num_of_citedby,citedby_authors,citedby_journals,article_abstract
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Why Cognitive Science,1977.0,1.0,1,1-2,January,10.1207/s15516709cog0101_1,Original Article,Allan Collins,...,NaN,NaN,NaN,0.0,NaN,"Marco Elio Tabacchi, Settimo Termini, “The hum...",12.0,Marco Elio Tabacchi/nSettimo Termini/n2017/nLo...,Biophysical Chemistry/nJournal of the Associat...,NaN
2,3,"An Overview of KRL, a Knowledge Representation...",1977.0,1.0,1,3-46,January,10.1207/s15516709cog0101_2,Original Article,"Daniel G. Bobrow1,* and\nTerry Winograd2",...,Artificial Intelligence/nCommunication of the ...,Human associative memory/nRepresentation and u...,"Human associative memory/nGus, a frame driven ...",1.0,"Gus, a frame driven dialog system/nA model and...","Orlando Lopez-Cruz, Nini Johanna Garnica, Tren...",278.0,Orlando Lopez-Cruz/nNini Johanna Garnica/n2018...,Computational Linguistics/nExpert Systems with...,"This paper describes KRL, a Knowledge Represen..."
3,4,Human and Computational Question Answering†,1977.0,1.0,1,47-73,January,10.1207/s15516709cog0101_3,Original Article,Wendy Lehnert*,...,American Journal of Computational Linguistics/...,Representation and understanding/nCenter for H...,Concepts for representing mundane reality in p...,0.0,An approach to the representation of mundane w...,"Colleen E. Crangle, Joyce Brothers Kart, A que...",31.0,Colleen E. Crangle/nJoyce Brothers Kart/n2015/...,PeerJ/nTheory and Practice in Language Studies...,The ability to answer questions about a text i...
4,5,Definite Descriptions and Semantic Memory†,1977.0,1.0,1,74-83,January,10.1207/s15516709cog0101_4,Original Article,Andrew Ortony* and\nRichard C. Anderson,...,Cognitive Psychology/nAmerican Educational Res...,Human associative memory/nTranslations from th...,Human associative memory/nIndividuation and re...,1.0,Individuation and reference in memory: Proper ...,"Kristine H. Onishi, Gregory L. Murphy, Discour...",7.0,Kristine H. Onishi/nGregory L. Murphy/n2002/nB...,Language and Cognitive Processes/nJournal of M...,Subjects were exposed to sentences containing ...
5,6,"Artificial Intelligence, Language, and the Stu...",1977.0,1.0,1,84-123,January,10.1207/s15516709cog0101_5,Original Article,Professor Ira Goldstein* and\nSeymour Papert,...,Linguistic Analysis/nArtificial Intelligence/n...,Human associative memory/nRepresentation and u...,Human associative memory/nProposal for knowled...,1.0,Questions of form and interpretation/nSummary ...,"Antoine C. M. van den Bogaart, Hans G. K. Humm...",37.0,Antoine C. M. van den Bogaart/nHans G. K. Humm...,Professional Development in Education/nArtific...,This paper studies the relationship of Artific...
6,7,Induction of Augmented Transition Networks†,1977.0,1.0,2,125-157,April,10.1207/s15516709cog0102_1,Original Article,John R. Anderson*,...,Information and Control/nJournal of Mathematic...,Language acquisition by computer and child/nIn...,Language acquisition by computer and child/nCo...,0.0,Language identification in the limit/nA mathem...,"Georgia M. Green, Non-Transformational Syntax,...",25.0,Georgia M. Green/n2011/nMichael Gasser/n2006/n...,Neural Networks/nLingua/nComputers & Industria...,LAS is a program that acquires augmented trans...
7,8,Decision Theory and Artificial Intelligence II...,1977.0,1.0,2,158-192,April,10.1207/s15516709cog0102_2,Original Article,"Jerome A. Feldman1,* and\nRobert F. Sproull2",...,Computing Surveys/nOperations Research/nArtifi...,Representation and understanding/nPhD thesis/n...,New programming languages for AI research/nAn ...,1.0,New programming languages for AI research/nSea...,"Nils Bulling, A Survey of Multi-Agent Decision...",24.0,Nils Bulling/n2014/nCharisma F. Choudhury/nMos...,KI - Künst

In [6]:
df_pf.rename()
tbl = df_pd[["year", 0, "article_name", "article_type"]]
#index_paperName = Table().with_columns(["Year",df.column(2),
#                                        "Paper Index", df.column(0),
#                                        "Article Name", df.column(1),
#                                        "Old Type", df.column(9)])

NameError: name 'df_pf' is not defined

In [ ]:
# create list of lists where the number in each sublist is the article index and the length of the 
# sublist is the number of authors for that article
# - i+1 to counter-act data indexing
df_namesplit = []
df_numauth = []
for i in range(1, len(df_ad)):
        
    temp = df_ad[i].split('/n')
    df_namesplit.append(temp)
    df_numauth.append([i+1]*len(temp))

#print(df_numauth)

In [4]:
# Makes list of lists into single list
finList = []
for i in range(len(df_numauth)):
    for j in range(len(df_numauth[i])):
        finList.append(df_numauth[i][j])
        
#print(finList)

NameError: name 'df_numauth' is not defined

In [ ]:
finList.to_csv("Num_Authors_Per_Article")

In [ ]:
# This function takes a string (author of article ) and strips "Corresponding Author" -> returns stripped string
def clearCorresponding(author):
    result = []
    if ',Corresponding author' in author:

        endIndex = author.find(',')
        result.append(author[:endIndex])
        return result
    
    elif 'Corresponding author' in author:
        endIndex = author.find('Corresponding author')
        
        result.append(author[:endIndex])
        return result
   
    else:
        return author

In [ ]:
# Main algorithm to construct table of Articles and Authors

results = Table(make_array("Article Id", "Author", "Department/University", "Corresponding Author"))
# For each sublist in the list of lists df_namesplit
articleCounter = 0 # Used to iterate through list of article id's
for i in range(len(df_namesplit)):
    
    articlesAuthors = df_namesplit[i]
    
    for j in range(len(articlesAuthors)):
        corr_auth = 0
        element = df_namesplit[i][j].split('             ')
        if len(element) > 1: # If the article has an author and the corresponding department/ university
    
            author = clearCorresponding(element[0])
            dept = element[1]
            
            if("Corresponding author" in df_namesplit[i][j]):
                corr_auth = 1
            results = results.with_row([finList[articleCounter],author, dept, corr_auth])
            articleCounter += 1
            
        else:            
            author = clearCorresponding(element[0])
            dept = "NaN"
            
            #newRow = make_array(corr_auth,author, dept)
            results = results.with_row([finList[articleCounter],author, dept, corr_auth])
            articleCounter += 1
            

In [ ]:
# Join article name with article id number
results = results.join("Article Id", index_paperName, "Paper Index")
# not sure why we need to drop column 5 twice
#results.show()

In [ ]:
Extended_Article = ["Extended Articles", "Extended Article", "Extended article", "Extended articles"]
Original_Article = ["Original Article", "Original Articles"]
Regular_Article = ["Regular Articles", "Regular articles", "Regular Article", "Regular article"]
Letter_to_Editor = ["Letters to the Editor","Letter to the Editor"]
Brief_Report = ["Brief Reports", "Brief Report", "Brief reports", "Brief report"]
Announcement = ["Announcement"]
Special = ["2011 Rumelhart Prize Special Issue Honoring Judea Pearl",
           "SPECIAL ISSUE: 2009 Rumelhart Prize Special Issue Honoring Susan Carey",
           "2010 Rumelhart Prize Special Issue Honoring James L. McClelland",
           "SPECIAL ISSUE: 2007 Rumelhart Prize Special Issue Honoring Jeffrey L. Elman Language as a Dynamical System"]
Comment = ["Comment","Short Communication"]
Commentary = ["Commentary"]


In [ ]:
def consolidate_Type (oldType):
    # put the definitions inside the function, but ideally in a separate data file
#     Extended_Article = ["Extended Articles", "Extended Article", "Extended article", "Extended articles"]
#     Original_Article = ["Original Article", "Original Articles"]
#     Regular_Article = ["Regular Articles", "Regular articles", "Regular Article", "Regular article"]
#     Letter_to_Editor = ["Letters to the Editor","Letter to the Editor"]
#     Brief_Report = ["Brief Reports", "Brief Report", "Brief reports", "Brief report"]
#     Announcement = ["Announcement"]
#     Special = ["2011 Rumelhart Prize Special Issue Honoring Judea Pearl",
#                "SPECIAL ISSUE: 2009 Rumelhart Prize Special Issue Honoring Susan Carey",
#                "2010 Rumelhart Prize Special Issue Honoring James L. McClelland",
#                "SPECIAL ISSUE: 2007 Rumelhart Prize Special Issue Honoring Jeffrey L. Elman Language as a Dynamical System"]
#     Comment = ["Comment","Short Communication"]
#     Commentary = ["Commentary"]
    if oldType in Extended_Article:
        return "Extended Article"
    if oldType in Original_Article:
        return "Original Article"
    if oldType in Regular_Article:
        return "Regular Article"
    if oldType in Brief_Report:
        return "Brief Report"
    if oldType in Letter_to_Editor:
        return "Letter to Editor"
    if oldType in Announcement:
        return "Announcement"
    if oldType in Special:
        return "Special"
    if oldType in Comment:
        return "Comment"
    if oldType in Commentary:
        return "Commentary"
    return "null"

In [ ]:
results = results.with_column("Article Type", results.apply(consolidate_Type, "Old Type"))
results = results.drop("Old Type").drop("Year_2").drop("Article Name_2")

In [ ]:
results = results.drop("Year_2").drop("Article Name_2").drop("New Type")
results

In [ ]:
results2 = Table().with_columns(["Article Id",[],
                                 "Author", [], 
                                 "Department/University", [], 
                                 "Corresponding Author",[],
                                 "Year",[],
                                 "Article Name", [],
                                 "New Type", [] ])


In [ ]:
for i in range(results.num_rows):
    if results.column(5)[i] != "null":
        results2 = results2.with_row(results.row(i))  
#results2.show()

In [ ]:
results2.to_csv("cleaned_data")

In [ ]:
# issue Computational Neuroscience
compSci = ["artificial intelligence", "computer science", "informatics", "computer", "cybernetics", "computing", "eecs", "technology"]

In [ ]:
def consolodateAffiliation(aff):
    temp = aff.lower()
    
    if "cognitive" in temp or "cognition" in temp:
        return "Cognitive Science"
    if "psychology" in temp or "psychological" in temp:
        return "Psychology"
    if "philosophy" in temp:
        return "Philosophy"
    if "linguistics" in temp:
        return "Linguistics"
    if "neuro" in temp or "brain" in temp:
        return "Neuroscience"
    for i in compSci:
        if i in temp:
            return "Artificial Intelligence"
    if "anthropology" in temp:
        return "Anthropology"
    if "social science" in temp:
        return "Social Science"
    return "Other"

In [ ]:
results2 = results2.with_column("Affiliation",results2.apply(consolodateAffiliation, 2))

In [ ]:
# HOW TO COUNT AFFILIATIONS
print(results2.where("Affiliation", are.equal_to("Cognitive Science")).num_rows)
print(results2.where("Affiliation", are.equal_to("Psychology")).num_rows)
print(results2.where("Affiliation", are.equal_to("Philosophy")).num_rows)
print(results2.where("Affiliation", are.equal_to("Artificial Intelligence")).num_rows)
print(results2.where("Affiliation", are.equal_to("Anthropology")).num_rows)
print(results2.where("Affiliation", are.equal_to("Neuroscience")).num_rows)
print(results2.where("Affiliation", are.equal_to("Linguistics")).num_rows)
print(results2.where("Affiliation", are.equal_to("Social Science")).num_rows)
print(results2.where("Affiliation", are.equal_to("Other")).num_rows)
#results2

In [ ]:
results2.where("Year", are.equal_to(1980)).sort("Article Name").show()

In [ ]:
keys = np.unique(results2['Affiliation'])
counts = np.zeros(len(keys))
print(keys)
for i,k in enumerate(keys):
    counts[i] = results2.where("Affiliation", are.equal_to(k)).num_rows

plt.figure(figsize=(10,10))
plt.bar(range(len(counts)),counts)
plt.xticks(range(len(counts)), keys, rotation=45);
#plt.ylim([0,500])

In [ ]:
# Artificial Intelligence
results2.where("Affiliation", are.equal_to("Artificial Intelligence")).hist("Year")

In [ ]:
# Psychology
results2.where("Affiliation", are.equal_to("Psychology")).hist("Year")

In [ ]:
# Philosophy
results2.where("Affiliation", are.equal_to("Philosophy")).hist("Year")

In [ ]:
results2.where("Affiliation", are.equal_to("Philosophy")).show()

In [ ]:
# Linguistics
results2.where("Affiliation", are.equal_to("Linguistics")).hist("Year")

In [ ]:
# Anthropology
results2.where("Affiliation", are.equal_to("Anthropology")).hist("Year")

In [ ]:
results2.where("Affiliation", are.equal_to("Anthropology")).show()

In [ ]:
# Neuroscience
results2.where("Affiliation", are.equal_to("Neuroscience")).hist("Year")

In [ ]:
# Logic, Language and Computation,
# Physics
# Decision technologies
# Neural computation
# Psycholinguistics Psycholinguistique
# Kinesiology
# department of speech pathology
# US ARMY Aeromedical Research
# Asian Studies
# Behaviorial Science
# Communication
# biophysics
# CECS
# Engineering
# Language Sciences
# Neural Science
# Scienza Cognitiva
# Department of Accounting and MIS
# Biological Sciences \n biology
# Electrical and Electronic Engineering
# Department of English Philology
# Department of Environmental Information
# Department of Hospital Medicine
# Department of Human Development
# Department of Humanities
# Department of Management Science and Information Systems
# Department of Management and Economics
# Department of Management and Organization
# Department of Mathematics
# Department of Organismic and Evolutionary Biology
# Department of Physical Medicine and Rehabilitation
# Department of Physical Therapy, University of Connecticut
# Department of Physiology, Feinberg School of Medicine
# Department of Psychiatry and the Behavioral Sciences
# Department of Radiology
# Department of Teaching, Learning, & Culture
# Discipline of Pharmacology
# Asian and African Area Studies

#classify departments into multiple bins
others = results2.where("Affiliation", are.equal_to("Other"))
others.select(2).sort(0,distinct=True).num_rows

In [ ]:
# All below cells are tests to see what the data is doing

In [ ]:
df_dept = pd.read_csv("cleaned_data")
colnames= ["Article Id","Author","Department_University","Corresponding Author","Article Name","Type"]
df_dept_new=pd.read_csv("cleaned_data", names=colnames)

In [ ]:
bad_inds = np.where(df_dept_new['Department_University'].isnull())[0]
len(bad_inds)
#len(df_dept_new)

In [ ]:
df_pd.iloc[368]['authors_and_details']

In [ ]:
df_dept_new.iloc[bad_inds]

In [ ]:
pd.set_option('display.max_rows', 100)
df_dept_new.groupby('Department_University',sort=True).Department_University.count().to_frame().sort_values('Department_University',ascending=False)

In [ ]:
# index the row (e.g. 2nd) and split by /n character
df['authors_and_details'][2].split('/n')

In [ ]:
# the 0th element contains the information of the first author
df['authors_and_details'][2].split('/n')[0]

In [ ]:
# further split author info to separate name and affiliation
au1 = df['authors_and_details'][2].split('/n')[0].split('             ')

In [ ]:
au1

In [ ]:
# find the corresponding author and delete that part of the string ',Corresponding author'
if 'Corresponding author' in au1[0]:
    endind = au1[0].find(',Corresponding author')
    print(endind)
    print(au1[0][:endind], '- ', au1[1])

In [ ]:
# make a new dataset


### The latter ones have a little more information
Let's go to index 1500

In [ ]:
df['authors_and_details'][1501].split('/n')

In [ ]:
df['article_name'][1501]